# Window Function

## row_number() , rank() & dense_rank()

In [2]:
#  SELECT
#      salary,
#      ROW_NUMBER()  OVER (ORDER BY salary) AS row_number,
#      RANK()        OVER (ORDER BY salary) AS rank,
#      DENSE_RANK()  OVER (ORDER BY salary) AS dense_rank
#  FROM employees
#  ORDER BY salary; 
 
#  SALARY | ROW_NUMBER | RANK | DENSE_RANK
#  1000   | 1          | 1    | 1
#  1500   | 2          | 2    | 2
#  1500   | 3          | 2    | 2
#  2000   | 4          | 4    | 3
#  2200   | 5          | 5    | 4
#  2500   | 6          | 6    | 5
#  2500   | 7          | 6    | 5
#  2500   | 8          | 6    | 5
#  3000   | 9          | 9    | 6

## Adding Partition by into Window Functions

In [1]:
#  SELECT
#      group_id   AS "GROUP",
#      salary     AS "SALARY",
#      ROW_NUMBER()  OVER (PARTITION BY group_id ORDER BY salary)  AS "ROW_NUMBER",
#      RANK()        OVER (PARTITION BY group_id ORDER BY salary)  AS "RANK",
#      DENSE_RANK()  OVER (PARTITION BY group_id ORDER BY salary)  AS "DENSE_RANK"
#  FROM employee_salaries
#  ORDER BY group_id, salary;
#  
#  
#  GROUP | SALARY | ROW_NUMBER | RANK | DENSE_RANK
#  A     | 1000   | 1          | 1    | 1
#  A     | 1500   | 2          | 2    | 2
#  A     | 1500   | 3          | 2    | 2
#  A     | 2000   | 4          | 4    | 3
#  B     | 2200   | 1          | 1    | 1
#  B     | 2500   | 2          | 2    | 2
#  B     | 2500   | 3          | 2    | 2
#  B     | 2500   | 4          | 2    | 2
#  B     | 3000   | 5          | 5    | 3

## FIRST_VALUE()

- Returns the first value in the window frame
    

In [3]:
#  FIRST_VALUE(salary)
#      OVER (PARTITION BY group_id ORDER BY salary)
#      
#  -- Default
#  RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW

- Default : From the first row of this partition up to the current row, give me the first salary.

## LAST_VALUE()

- By default ; From the first row up to the current row, give me the last salary - So it just equals the current row’s salary ( useless most of the time)

- From the very first row to the very last row in this partition, give me the last salary

In [4]:
#  LAST_VALUE(salary)
#     OVER (
#       PARTITION BY group_id
#       ORDER BY salary
#       ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
#     )

## Getting the cumulative value - UNBOUNDED PRECEDING AND CURRENT ROW

In [ ]:
# SUM(salary) OVER (
#   PARTITION BY group_id
#   ORDER BY salary
#   ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
# ) AS running_sum

## Getting Value For a Partition : UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING

In [ ]:
#  FIRST_VALUE(salary) OVER (
#   PARTITION BY group_id
#   ORDER BY salary
#   ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
# ) AS min_salary_in_group,
# 
# LAST_VALUE(salary) OVER (
#   PARTITION BY group_id
#   ORDER BY salary
#   ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
# ) AS max_salary_in_group

## Sliding Window around current row

In [ ]:
# ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING

In [ ]:
#  AVG(salary) OVER (
#    PARTITION BY group_id
#    ORDER BY salary
#    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
#  ) AS moving_avg

## LAG() & LEAD()

Give me the salary from 1 row before this one in the same partition

```sql
LAG(salary, 1) OVER (
  PARTITION BY group_id
  ORDER BY salary
)
```

Give me the salary from 1 row after this one in the same partition

```sql
LEAD(salary, 1) OVER (
  PARTITION BY group_id
  ORDER BY salary
)
```

## NTH_VALUE()

```sql
-- from the entire partition give me the second salary 
NTH_VALUE(salary, 2)
OVER (
  PARTITION BY group_id
  ORDER BY salary
  ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
)
```

## NTILE()

```sql
SELECT
  salary,
  NTILE(4) OVER (ORDER BY salary) AS tile
FROM employee_salaries
ORDER BY salary;
```

```sql
salary | tile
1000   | 1
1500   | 1
1500   | 1
2000   | 2
2200   | 2
2500   | 3
2500   | 3
2500   | 4
3000   | 4
```

# Case Statements

```sql
SELECT
    salary,
    CASE
        WHEN salary < 1500 THEN 'Low'
        WHEN salary >= 1500 AND salary < 2500 THEN 'Medium'
        ELSE 'High'
    END AS salary_category
FROM employee_salaries
ORDER BY salary;
```

```sql
salary | salary_category
1000   | Low
1500   | Medium
1500   | Medium
2000   | Medium
2200   | Medium
2500   | High
2500   | High
2500   | High
3000   | High
```

```sql
SELECT
    salary,
    CASE
        WHEN salary < 1500 THEN 'Low'
        WHEN salary >= 1500 AND salary < 2500 THEN 'Medium'
        ELSE 'High'
    END AS salary_category
FROM employee_salaries
ORDER BY salary;
```

```sql
salary | salary_category
1000   | Low
1500   | Medium
1500   | Medium
2000   | Medium
2200   | Medium
2500   | High
2500   | High
2500   | High
3000   | High
```

```sql
SELECT
    salary,
    SUM(salary) OVER (ORDER BY salary ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_sum,
    CASE
        WHEN SUM(salary) OVER (ORDER BY salary ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) < 3000 THEN 'Low Sum'
        WHEN SUM(salary) OVER (ORDER BY salary ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) BETWEEN 3000 AND 6000 THEN 'Medium Sum'
        ELSE 'High Sum'
    END AS sum_category
FROM employee_salaries
ORDER BY salary;
```

```sql
salary | running_sum | sum_category
1000   | 1000        | Low Sum
1500   | 2500        | Low Sum
1500   | 4000        | Medium Sum
2000   | 6000        | Medium Sum
2200   | 8200        | High Sum
2500   | 10700       | High Sum
3000   | 13700       | High Sum
```

```sql
SELECT
    COUNT(CASE WHEN salary < 2000 THEN 1 END) AS low_count,
    COUNT(CASE WHEN salary >= 2000 THEN 1 END) AS high_count
FROM employee_salaries;

```

```sql
low_count | high_count
3         | 5
```

# Nice Questions

## Company wants to find the Top-paid employee in each department

```sql
CREATE TABLE emp_q1 (
    emp_id SERIAL PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    department VARCHAR(50),
    salary NUMERIC(10,2),
    join_date DATE
);

INSERT INTO emp_q1 (first_name, last_name, department, salary, join_date) VALUES
('Alice','Smith','Engineering',70000,'2021-03-15'),
('Bob','Johnson','Marketing',55000,'2019-07-22'),
('Carol','Williams','Engineering',95000,'2015-01-12'),
('David','Brown','Sales',50000,'2022-11-01'),
('Eve','Davis','Marketing',60000,'2020-05-18'),
('Frank','Miller','Management',120000,'2010-06-30'),
('Grace','Wilson','Sales',52000,'2018-09-10');

SELECT emp_id, first_name, last_name, department, salary, join_date
FROM (
    SELECT *,
           RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS dept_rank
    FROM emp_q1
) sub
WHERE dept_rank = 1;
```

## Sales company wants to know monthly sales per employee, but only include employees whose monthly sales exceed the department average.

In [ ]:
# CREATE TABLE emp_q2 (
#     emp_id SERIAL PRIMARY KEY,
#     first_name VARCHAR(50),
#     last_name VARCHAR(50),
#     department VARCHAR(50)
# );
# 
# INSERT INTO emp_q2 (first_name, last_name, department) VALUES
# ('Alice','Smith','Sales'),
# ('Bob','Johnson','Sales'),
# ('Carol','Williams','Sales'),
# ('David','Brown','Marketing'),
# ('Eve','Davis','Marketing');
# 
# CREATE TABLE sales_q2 (
#     sale_id SERIAL PRIMARY KEY,
#     emp_id INT,
#     sale_amount NUMERIC(10,2),
#     sale_date DATE
# );
# 
# INSERT INTO sales_q2 (emp_id, sale_amount, sale_date) VALUES
# (1, 1000, '2026-01-01'),
# (1, 2000, '2026-01-15'),
# (2, 1500, '2026-01-10'),
# (3, 2500, '2026-01-20'),
# (4, 3000, '2026-01-05'),
# (5, 3500, '2026-01-25');
# 
# WITH emp_monthly AS (
#     SELECT e.emp_id, e.first_name, e.department,
#            DATE_TRUNC('month', s.sale_date) AS month,
#            SUM(s.sale_amount) AS total_sales
#     FROM emp_q2 e
#     JOIN sales_q2 s ON e.emp_id = s.emp_id
#     GROUP BY e.emp_id, e.first_name, e.department, DATE_TRUNC('month', s.sale_date)
# ),
# dept_avg AS (
#     SELECT department, month, AVG(total_sales) AS avg_sales
#     FROM emp_monthly
#     GROUP BY department, month
# )
# SELECT em.emp_id, em.first_name, em.department, em.month, em.total_sales
# FROM emp_monthly em
# JOIN dept_avg da
#   ON em.department = da.department AND em.month = da.month
# WHERE em.total_sales > da.avg_sales;
